<a href="https://colab.research.google.com/github/Apsod/DD2360HT22/blob/main/hw_2/ex_4/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://www.cs.virginia.edu/~skadron/lava/Rodinia/Packages/rodinia_3.1.tar.bz2
!tar -xf rodinia_3.1.tar.bz2

--2022-11-28 20:53:26--  http://www.cs.virginia.edu/~skadron/lava/Rodinia/Packages/rodinia_3.1.tar.bz2
Resolving www.cs.virginia.edu (www.cs.virginia.edu)... 128.143.67.11
Connecting to www.cs.virginia.edu (www.cs.virginia.edu)|128.143.67.11|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cs.virginia.edu/~skadron/lava/Rodinia/Packages/rodinia_3.1.tar.bz2 [following]
--2022-11-28 20:53:27--  https://www.cs.virginia.edu/~skadron/lava/Rodinia/Packages/rodinia_3.1.tar.bz2
Connecting to www.cs.virginia.edu (www.cs.virginia.edu)|128.143.67.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 360636246 (344M) [application/x-tar]
Saving to: ‘rodinia_3.1.tar.bz2’

rodinia_3.1.tar.bz2 100%[===================>] 343.93M  12.6MB/s    in 29s     

2022-11-28 20:53:58 (11.8 MB/s) - ‘rodinia_3.1.tar.bz2’ saved [360636246/360636246]



In [2]:
TXT= """
CUDA_DIR = /usr/local/cuda
CUDA_LIB_DIR = ${CUDA_DIR}/lib64
SDK_DIR = ${CUDA_DIR}/samples/
""".strip()

with open('rodinia_3.1/common/make.config', 'wt') as handle:
  handle.write(
    TXT      
  )

In [6]:
from pathlib import Path 
import subprocess
import itertools

ROOT = Path('.').absolute()

implementations = ['cuda', 'openmp']
problems = ['backprop', 'nw', 'nn']
for impl, problem in itertools.product(implementations, problems):
  path = ROOT / 'rodinia_3.1' / impl / problem
  print(f'\n\n================\n\nmaking {impl} {problem}\n{path}')
  p = subprocess.run(f"cd {path}; make -B", shell=True, capture_output=True)
  print(p.stdout.decode('utf-8'))
  print(p.stderr.decode('utf-8'))




making cuda backprop
/content/rodinia_3.1/cuda/backprop
gcc -g  -O2 backprop.c -c
gcc -g  -O2 facetrain.c -c
gcc -g  -O2 imagenet.c -c
/usr/local/cuda/bin/nvcc -I/usr/local/cuda/include -O2 -c backprop_cuda.cu
gcc -g  -O2 backprop.o facetrain.o imagenet.o backprop_cuda.o -o backprop -L/usr/local/cuda/lib64 -lcuda -lcudart -lm

backprop.c:92:1: warning: return type defaults to ‘int’ [-Wimplicit-int]
 bpnn_randomize_weights(w, m, n)
 ^~~~~~~~~~~~~~~~~~~~~~
backprop.c:106:1: warning: return type defaults to ‘int’ [-Wimplicit-int]
 bpnn_randomize_row(w, m)
 ^~~~~~~~~~~~~~~~~~
backprop.c:118:1: warning: return type defaults to ‘int’ [-Wimplicit-int]
 bpnn_zero_weights(w, m, n)
 ^~~~~~~~~~~~~~~~~
backprop.c: In function ‘bpnn_initialize’:
backprop.c:132:6: warning: type of ‘seed’ defaults to ‘int’ [-Wimplicit-int]
 void bpnn_initialize(seed)
      ^~~~~~~~~~~~~~~
backprop.c: In function ‘bpnn_adjust_weights’:
backprop.c:305:6: warning: type of ‘ndelta’ defaults to ‘int’ [-Wimplicit-int]
 

In [7]:
# runmap for varying the sizes: making the neural net in backprop larger,
# and the sequences in Needleman-Wunsch longer. 

runmap = {
    ('cuda', 'backprop'): './backprop 524288',
    ('openmp', 'backprop'): './backprop 524288',
    ('cuda', 'nw'): './needle 8192 10',
    ('openmp', 'nw'): './needle 8192 10 2',
    ('cuda', 'nn'): './run',
    ('openmp', 'nn'): './run',
}

In [8]:
from time import perf_counter

for problem, impl in itertools.product(problems, implementations):
  path = ROOT / 'rodinia_3.1' / impl / problem
  print(f'\n=======\nrunning {impl} {problem} @ {path}')
  start = perf_counter()
  p = subprocess.run(f"cd {path}; {runmap[(impl, problem)]}", shell=True, capture_output=True)
  stop = perf_counter()
  print(f'elapsed time: {stop - start} seconds')
  print('STDOUT')
  print(p.stdout.decode('utf-8'))
  print('STDERR')
  print(p.stderr.decode('utf-8'))




running cuda backprop @ /content/rodinia_3.1/cuda/backprop
elapsed time: 0.8414313719999882 seconds
STDOUT
Random number generator seed: 7
Input layer size : 524288
Starting training kernel
Performing GPU computation
Training done

STDERR


running openmp backprop @ /content/rodinia_3.1/openmp/backprop
elapsed time: 0.5598553029999493 seconds
STDOUT
Random number generator seed: 7
Input layer size : 524288
Starting training kernel
Performing CPU computation
Training done

STDERR


running cuda nw @ /content/rodinia_3.1/cuda/nw
elapsed time: 1.2709616020000567 seconds
STDOUT
WG size of kernel = 16 
Start Needleman-Wunsch
Processing top-left matrix
Processing bottom-right matrix

STDERR


running openmp nw @ /content/rodinia_3.1/openmp/nw
elapsed time: 0.7006740920000993 seconds
STDOUT
Start Needleman-Wunsch
Num of threads: 2
Processing top-left matrix
Processing bottom-right matrix
Total time: 0.373 seconds

STDERR


running cuda nn @ /content/rodinia_3.1/cuda/nn
elapsed time: 0.277117